In [268]:
import torch
from typing import Optional, Tuple

In [269]:
seq_len = 8
model_dim = 6

In [270]:
position = torch.arange(seq_len)
position.shape

torch.Size([8])

In [271]:
even_i = torch.arange(0, model_dim, 2)
even_i.shape

torch.Size([3])

In [272]:
odd_i = torch.arange(1, model_dim, 2)
odd_i.shape

torch.Size([3])

In [273]:
odd_pe = torch.cos(torch.outer(position, 1.0 / (10000 ** (odd_i / model_dim))))
odd_pe.size()

torch.Size([8, 3])

In [274]:
even_pe = torch.sin(torch.outer(position, 1.0 / (10000 ** (even_i / model_dim))))
even_pe.size()

torch.Size([8, 3])

In [275]:
odd_pe

tensor([[1.0000, 1.0000, 1.0000],
        [0.9769, 0.9999, 1.0000],
        [0.9086, 0.9998, 1.0000],
        [0.7983, 0.9996, 1.0000],
        [0.6511, 0.9992, 1.0000],
        [0.4738, 0.9988, 1.0000],
        [0.2746, 0.9982, 1.0000],
        [0.0627, 0.9976, 1.0000]])

In [276]:
even_pe

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.8415,  0.0464,  0.0022],
        [ 0.9093,  0.0927,  0.0043],
        [ 0.1411,  0.1388,  0.0065],
        [-0.7568,  0.1846,  0.0086],
        [-0.9589,  0.2300,  0.0108],
        [-0.2794,  0.2749,  0.0129],
        [ 0.6570,  0.3192,  0.0151]])

In [277]:
stacked = torch.stack([even_pe, odd_pe], dim=2)
stacked

tensor([[[ 0.0000,  1.0000],
         [ 0.0000,  1.0000],
         [ 0.0000,  1.0000]],

        [[ 0.8415,  0.9769],
         [ 0.0464,  0.9999],
         [ 0.0022,  1.0000]],

        [[ 0.9093,  0.9086],
         [ 0.0927,  0.9998],
         [ 0.0043,  1.0000]],

        [[ 0.1411,  0.7983],
         [ 0.1388,  0.9996],
         [ 0.0065,  1.0000]],

        [[-0.7568,  0.6511],
         [ 0.1846,  0.9992],
         [ 0.0086,  1.0000]],

        [[-0.9589,  0.4738],
         [ 0.2300,  0.9988],
         [ 0.0108,  1.0000]],

        [[-0.2794,  0.2746],
         [ 0.2749,  0.9982],
         [ 0.0129,  1.0000]],

        [[ 0.6570,  0.0627],
         [ 0.3192,  0.9976],
         [ 0.0151,  1.0000]]])

In [278]:
pe = torch.flatten(stacked, start_dim=1, end_dim=2)
pe

tensor([[ 0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
        [ 0.8415,  0.9769,  0.0464,  0.9999,  0.0022,  1.0000],
        [ 0.9093,  0.9086,  0.0927,  0.9998,  0.0043,  1.0000],
        [ 0.1411,  0.7983,  0.1388,  0.9996,  0.0065,  1.0000],
        [-0.7568,  0.6511,  0.1846,  0.9992,  0.0086,  1.0000],
        [-0.9589,  0.4738,  0.2300,  0.9988,  0.0108,  1.0000],
        [-0.2794,  0.2746,  0.2749,  0.9982,  0.0129,  1.0000],
        [ 0.6570,  0.0627,  0.3192,  0.9976,  0.0151,  1.0000]])

In [279]:
def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    """
    Precompute the frequency tensor for complex exponentials (cis) with given dimensions.

    This function calculates a frequency tensor with complex exponentials using the given dimension 'dim'
    and the end index 'end'. The 'theta' parameter scales the frequencies.
    The returned tensor contains complex values in complex64 data type.

    Args:
        dim (int): Dimension of the frequency tensor.
        end (int): End index for precomputing frequencies.
        theta (float, optional): Scaling factor for frequency computation. Defaults to 10000.0.

    Returns:
        torch.Tensor: Precomputed frequency tensor with complex exponentials.

    
        

    """
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device)  # type: ignore
    freqs = torch.outer(t, freqs).float()  # type: ignore
    freqs_cis = torch.polar(torch.ones_like(freqs), freqs)  # complex64
    return freqs_cis

In [280]:
freqs_cis = precompute_freqs_cis(6, 4)
freqs_cis.shape

torch.Size([4, 3])

In [281]:
def reshape_for_broadcast(freqs_cis: torch.Tensor, x: torch.Tensor):
    """
    Reshape frequency tensor for broadcasting it with another tensor.

    This function reshapes the frequency tensor to have the same shape as the target tensor 'x'
    for the purpose of broadcasting the frequency tensor during element-wise operations.

    Args:
        freqs_cis (torch.Tensor): Frequency tensor to be reshaped.
        x (torch.Tensor): Target tensor for broadcasting compatibility.

    Returns:
        torch.Tensor: Reshaped frequency tensor.

    Raises:
        AssertionError: If the frequency tensor doesn't match the expected shape.
        AssertionError: If the target tensor 'x' doesn't have the expected number of dimensions.
    """
    print("freqs_cis before:",freqs_cis.shape)
    ndim = x.ndim
    assert 0 <= 1 < ndim
    assert freqs_cis.shape == (x.shape[1], x.shape[-1])
    print("x.shape: ",x.shape)
    shape = [d if i == 1 or i == ndim - 1 else 1 for i, d in enumerate(x.shape)]
    print("after:",shape)
    return freqs_cis.view(*shape)

In [282]:
def apply_rotary_emb(
    xq: torch.Tensor,
    xk: torch.Tensor,
    freqs_cis: torch.Tensor,
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Apply rotary embeddings to input tensors using the given frequency tensor.

    This function applies rotary embeddings to the given query 'xq' and key 'xk' tensors using the provided
    frequency tensor 'freqs_cis'. The input tensors are reshaped as complex numbers, and the frequency tensor
    is reshaped for broadcasting compatibility. The resulting tensors contain rotary embeddings and are
    returned as real tensors.

    Args:
        xq (torch.Tensor): Query tensor to apply rotary embeddings.
        xk (torch.Tensor): Key tensor to apply rotary embeddings.
        freqs_cis (torch.Tensor): Precomputed frequency tensor for complex exponentials.

    Returns:
        Tuple[torch.Tensor, torch.Tensor]: Tuple of modified query tensor and key tensor with rotary embeddings.

        

    """
    print("xq reshape:",xq.float().reshape(*xq.shape[:-1], -1, 2).shape)
    xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2))
    print("xq_.shape:",xq_.shape)
    xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2))
    print("xk_.shape:",xk_.shape)
    freqs_cis = reshape_for_broadcast(freqs_cis, xq_)
    print("freqs_cis.shape:",freqs_cis.shape)
    xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(3)
    xk_out = torch.view_as_real(xk_ * freqs_cis).flatten(3)
    print("xq_out,xk_out shape:",xq_out.shape,xk_out.shape)
    return xq_out.type_as(xq), xk_out.type_as(xk)

In [283]:
a = torch.tensor([[1.0, 2.0],
                  [3.0, 4.0]])  # shape [2, 2]
torch.view_as_complex(a)

tensor([1.+2.j, 3.+4.j])

In [284]:
# (batch_size, seq_len, num_heads, head_dim)
xq = torch.randn(1,4,2,6)
xk = torch.randn(1,4,2,6)
xq,xk = apply_rotary_emb(xq,xk,freqs_cis)

xq reshape: torch.Size([1, 4, 2, 3, 2])
xq_.shape: torch.Size([1, 4, 2, 3])
xk_.shape: torch.Size([1, 4, 2, 3])
freqs_cis before: torch.Size([4, 3])
x.shape:  torch.Size([1, 4, 2, 3])
after: [1, 4, 1, 3]
freqs_cis.shape: torch.Size([1, 4, 1, 3])
xq_out,xk_out shape: torch.Size([1, 4, 2, 6]) torch.Size([1, 4, 2, 6])
